# Analytic Vidya - Black Friday 

A retail company “ABC Private Limited” wants to understand the customer purchase behaviour (specifically, purchase amount) against various products of different categories. They have shared purchase summary of various customers for selected high volume products from last month.
The data set also contains customer demographics (age, gender, marital status, city_type, stay_in_current_city), product details (product_id and product category) and Total purchase_amount from last month.
Now, they want to build a model to predict the purchase amount of customer against various products which will help them to create personalized offer for customers against different products.

## Data

| Variable                   | Definition                                          |
|----------------------------|-----------------------------------------------------|
| User_ID                    | User ID                                             |
| Product_ID                 | Product ID                                          |
| Gender                     | Sex of User                                         |
| Age                        | Age in bins                                         |
| Occupation                 | Occupation (Masked)                                 |
| City_Category              | Category of the City (A,B,C)                        |
| Stay_In_Current_City_Years | Number of years stay in current city                |
| Marital_Status             | Marital Status                                      |
| Product_Category_1         | Product Category (Masked)                           |
| Product_Category_2         | Product may belongs to other category also (Masked) |
| Product_Category_3         | Product may belongs to other category also (Masked) |
| Purchase                   | Purchase Amount (Target Variable)                   |

## Exploration

In [1]:
import pandas as pd

In [2]:
filename = './train_oSwQCTC/train.csv'
df = pd.read_csv(filename)

In [3]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [4]:
# missing data in Product_Category_2 and Product_Category_3
df.count()

User_ID                       550068
Product_ID                    550068
Gender                        550068
Age                           550068
Occupation                    550068
City_Category                 550068
Stay_In_Current_City_Years    550068
Marital_Status                550068
Product_Category_1            550068
Product_Category_2            376430
Product_Category_3            166821
Purchase                      550068
dtype: int64

In [5]:
# there are mutiple purchase per user in this dataset
df.User_ID.duplicated().any()

True

In [6]:
# How many users?
df.User_ID.value_counts().count()

5891

In [7]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

_ , axPur = plt.subplots()
# What is the distribution of purchases price?
df.Purchase.hist(color='b', edgecolor ='k', ax=axPur)

<IPython.core.display.Javascript object>

In [8]:
_ , axUID = plt.subplots()
# What is the average purchase amount per user?
df.groupby(['User_ID']).Purchase.mean().hist(ax=axUID)

<IPython.core.display.Javascript object>

In [9]:
_ , axUIDCnt = plt.subplots()
# how many purchase instance per user ?
df.groupby(['User_ID']).User_ID.count().hist(bins=1000, ax=axUIDCnt)
# axUIDCnt.set_xlim(0,100)
# There is a peak around 16 purchases/user

<IPython.core.display.Javascript object>

## Gender

There are only Male or Female in this dataset, encode with arbitrary labeler (i.e. 0 and 1)

In [10]:
# how many different gender types?
df.Gender.value_counts()

M    414259
F    135809
Name: Gender, dtype: int64

In [11]:
# use scikit-learn encoder
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
Gender_encoder = LabelEncoder()
Gender_coded = Gender_encoder.fit_transform(df.Gender.values)
Gender_encoder.classes_
# sparse matrix conversion
Gender_coded = csr_matrix(Gender_coded.reshape(-1,1))

In [12]:
# ENCODING OF GENDER IN PANDAS:
# reload data for debug
# df = pd.read_csv(filename)

## gives SettingWithCopyWarning (double indexing, Gender first then boolean) but still works by chance
## 84.8 ms
# %timeit df.Gender[df.Gender == 'M'] = 0
# df.Gender[df.Gender == 'F'] = 1
## equivalent, gives SettingWithCopyWarning but works
## 99.4 ms
# %timeit df.loc[:,'Gender'][df.Gender == 'M'] = 0
# df.loc[:,'Gender'][df.Gender == 'F'] = 1

# The following statements produces SettingWithCopyWarning AND doesn't do the assigment
# typical chain indexing, meaning index twice. Make an assignment on a copy in this case!
# 95.8 ms
# %timeit df[df.Gender == 'M'].loc[:,'Gender'] = 0
# df[df.Gender == 'F'].loc[:,'Gender'] = 1

## works, no warning but not the fastest (avoid using apply because it is slow in general)
## 82.2 ms / 2 operations = 41.1 ms 
# %timeit df.loc[:,'Gender'] = df.loc[:,'Gender'].apply(lambda x: 0 if x == 'M' else 1)

# Correct solution single indexing, it works and fastest solution
# 20 ms !!! (faster than using sklearn encoder)
# %timeit df.loc[df.Gender == 'M', 'Gender'] = 0
# df.loc[df.Gender == 'F', 'Gender'] = 1

In [13]:
df.Gender.head()

0    F
1    F
2    F
3    F
4    M
Name: Gender, dtype: object

## Age
There are age categories, encode them via one-hot encoding scheme

In [14]:
# how many different age categories?
df.Age.value_counts()

26-35    219587
36-45    110013
18-25     99660
46-50     45701
51-55     38501
55+       21504
0-17      15102
Name: Age, dtype: int64

In [15]:
# use scikit-learn encoder
from sklearn.preprocessing import LabelBinarizer
Age_encoder = LabelBinarizer( sparse_output=True )
Age_coded = Age_encoder.fit_transform(df.Age.values)

# print all age categories
print(Age_encoder.classes_)
# print associated encoded values
print(Age_encoder.fit_transform(Age_encoder.classes_))
# resulting feature array should have as many columns as age categories
print(Age_coded.shape)

['0-17' '18-25' '26-35' '36-45' '46-50' '51-55' '55+']
  (0, 0)	1
  (1, 1)	1
  (2, 2)	1
  (3, 3)	1
  (4, 4)	1
  (5, 5)	1
  (6, 6)	1
(550068, 7)


## Occupation
Occupation are masked. In other words they are already encoded.<br>
Occupation are categorical so we need to use one-hot encoding.

In [16]:
# what are the different occupation ?
df.Occupation.value_counts().sort_index()

0     69638
1     47426
2     26588
3     17650
4     72308
5     12177
6     20355
7     59133
8      1546
9      6291
10    12930
11    11586
12    31179
13     7728
14    27309
15    12165
16    25371
17    40043
18     6622
19     8461
20    33562
Name: Occupation, dtype: int64

In [17]:
from sklearn.preprocessing import OneHotEncoder
# outputs sparse matrix by default
Occ_encoder = OneHotEncoder()
Occ_coded = Occ_encoder.fit_transform(df.Occupation.values.reshape(-1,1))

#  number of different values per feature (i.e. per column) encoder found 
print(Occ_encoder.n_values_ )
# resulting feature array should have as many columns as categories
print(Occ_coded.shape)

[21]
(550068, 21)


## City categories
There are 3 city categories: A, B and C.<br>
Encoded them via labelbinarizer

In [18]:
# what are the different city categories?
df.City_Category.value_counts().sort_index()

A    147720
B    231173
C    171175
Name: City_Category, dtype: int64

In [19]:
# use scikit-learn encoder
from sklearn.preprocessing import LabelBinarizer
City_encoder = LabelBinarizer(sparse_output=True)
City_coded = City_encoder.fit_transform(df.City_Category.values)

# print all categories
print(City_encoder.classes_)
# print associated encoded values
print(City_encoder.fit_transform(City_encoder.classes_))
# resulting feature array should have as many columns as categories
print(City_coded.shape)

['A' 'B' 'C']
  (0, 0)	1
  (1, 1)	1
  (2, 2)	1
(550068, 3)


## Number of year in current city
You can select 0 to 3 years then it is 4+ years.<br>
Replace 4+ with 4 years to keep only numeric values. All we know is it has been at least 4 years.


In [20]:
# years in current city
df.Stay_In_Current_City_Years.value_counts().sort_index()

0      74398
1     193821
2     101838
3      95285
4+     84726
Name: Stay_In_Current_City_Years, dtype: int64

In [21]:
from scipy.sparse import csr_matrix
Years_Current_City = df.loc[:,'Stay_In_Current_City_Years'].apply(lambda x: 4 if x == '4+' else int(x)).values
Years_Current_City_csr = csr_matrix(Years_Current_City.reshape(-1,1))
type(Years_Current_City)

numpy.ndarray

In [22]:
# try polynomes on years in current city
from sklearn.preprocessing import PolynomialFeatures
from scipy.sparse import csr_matrix
# object to compute
poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
# compute polynomial features
Years_Current_City_poly = poly.fit_transform(Years_Current_City.reshape(-1,1))
# convert to sparse matrix
Years_Current_City_polycsr = csr_matrix(Years_Current_City_poly)
Years_Current_City_polycsr.shape

(550068, 3)

## Marital status
Marital status is 0 or 1. I'm guessing it means you're married if 1 otherwise you're not with 0.<br>
Can be used as given (i.e. without modifications/encoding).

In [23]:
# categories of marital status
df.Marital_Status.value_counts().sort_index()

0    324731
1    225337
Name: Marital_Status, dtype: int64

In [24]:
from scipy.sparse import csr_matrix
Mar_status = df.Marital_Status.values
Mar_status = csr_matrix(Mar_status.reshape(-1,1))
Mar_status.shape

(550068, 1)

## Product Categories
Category 1 has 20 products categories (1 to 20). <br>
Category 2 has 17 products categories (2.0 to 18.0). <br>
Category 3 has 15 products categories (3.0 to 18.0, missing 7.0).

In [25]:
# categories of marital status
df.Product_Category_1.value_counts().sort_index()

1     140378
2      23864
3      20213
4      11753
5     150933
6      20466
7       3721
8     113925
9        410
10      5125
11     24287
12      3947
13      5549
14      1523
15      6290
16      9828
17       578
18      3125
19      1603
20      2550
Name: Product_Category_1, dtype: int64

In [26]:
from sklearn.preprocessing import OneHotEncoder
Prod1_encoder = OneHotEncoder()
Prod1_coded = Prod1_encoder.fit_transform(df.Product_Category_1.values.reshape(-1,1))

#  number of different values per feature (i.e. per column) encoder found 
print(Prod1_encoder.n_values_ )
# resulting feature array should have as many columns as categories
print(Prod1_coded.shape)

[21]
(550068, 20)


Product Category 1 is good to go. Let's see if we can combine all product categories

In [29]:
df.loc[:3, 'Product_Category_1'].apply(type)

0    <class 'int'>
1    <class 'int'>
2    <class 'int'>
3    <class 'int'>
Name: Product_Category_1, dtype: object

In [28]:
# combine Prod_1, Prod_2 and Prod_3
temp = df.apply(lambda x:\
            '-'.join([str(x['Product_Category_1']),\
                      str(x['Product_Category_2']), \
                      str(x['Product_Category_3']), \
                     ]), axis=1 )
# name of column in newly created pandas series
temp.name='Product_Cat123'
#'-'.join([str(df.loc[2, 'Product_Category_1']), str(df.loc[2, 'Product_Category_2']) ] )
# could convert to string first with
#df.loc[:,['col1', 'col2']].astype(str)

KeyboardInterrupt: 

In [ ]:
# number of different (Prod1-Prod2-Prod3) combination
temp.value_counts().count()

In [ ]:
# use scikit-learn encoder
from sklearn.preprocessing import LabelBinarizer
Prod_encoder = LabelBinarizer(sparse_output=True)
Prod_coded = Prod_encoder.fit_transform(temp.values)

# print all categories
print(Prod_encoder.classes_[:5])
# print associated encoded values
print(Prod_encoder.fit_transform(Prod_encoder.classes_)[:5])
# resulting feature array should have as many columns as categories
print(Prod_coded.shape)

## User ID

In [ ]:
# use scikit-learn encoder
from sklearn.preprocessing import LabelBinarizer
User_encoder = LabelBinarizer(sparse_output=True)
User_coded = User_encoder.fit_transform(df.User_ID.values)

# print all categories
print(User_encoder.classes_[:5])
# print associated encoded values
print(User_encoder.fit_transform(User_encoder.classes_)[:5])
# resulting feature array should have as many columns as categories
print(User_coded.shape)

# users are already encoded, only need one-hot encoding
#TODO

## Product ID

In [ ]:
# number of different products
df.Product_ID.value_counts().count()

In [ ]:
# use scikit-learn encoder
from sklearn.preprocessing import LabelBinarizer
ProdID_encoder = LabelBinarizer(sparse_output=True)
ProdID_coded = ProdID_encoder.fit_transform(df.Product_ID.values)

# print all categories
print(ProdID_encoder.classes_[:5])
# print associated encoded values
print(ProdID_encoder.fit_transform(ProdID_encoder.classes_)[:5])
# resulting feature array should have as many columns as categories
print(ProdID_coded.shape)

### Does each Product ID refer to a unique product category?

In [ ]:
# Group by product ID and see if each Product ID occurrence has a unique Product category triplet
df_ProdIDCat = pd.concat([df.loc[:,'Product_ID'], temp], axis=1)
df_ProdIDCat.count()

In [ ]:
# this produces an empty dataframe as they are no remaining data (only two columns)
# BUT index has tuples and if the first element repeats in more than one tuples then they are Product ID that fits
# in different category triplet
s = df_ProdIDCat.groupby(['Product_ID', 'Product_Cat123']).count()

# group by Prod ID then count the different Prod cat triplet for each Prod ID
s = df_ProdIDCat.groupby(['Product_ID']).Product_Cat123.value_counts()
# compare size of first level index (size of Prod ID returns the number of unique ProdID)
# to second level index (Prod Cat triplet, get all values as list and use length so that it doesn't only count unique values)
print(s.index.levels[0].size)
print(len(s.index.get_level_values(1)))
# NOTE: get_level_values()  returns a Int64Index object, len() and .shape work on it. List uses len(), numpy array uses .shape

# # EXTRA: filter df_ProdIDCat by product (too slow!)
# for ind, ID in df.Product_ID.value_counts().iteritems():
#     if df_ProdIDCat[df_ProdIDCat.Product_ID == ID].Product_Cat123.value_counts().size > 1:
#         print(ind,ID)

Conclusion is each product ID belongs to a unique Prod Cat123

## Combination of Age and Marital Status and City Category

In [ ]:
temp_Age_Marital_City = df.apply(lambda x:\
            '-'.join([str(x['Age']),\
                      str(x['Marital_Status']), \
                      str(x['City_Category']), \
                     ]), axis=1 )

In [ ]:
temp_Age_Marital_City.value_counts()

In [ ]:
# use scikit-learn encoder
from sklearn.preprocessing import LabelBinarizer
Age_Marital_City_encoder = LabelBinarizer(sparse_output=True)
Age_Marital_City_coded = Age_Marital_City_encoder.fit_transform(temp_Age_Marital_City.values)

# Linear Regression


In [ ]:
df.head(2)

In [ ]:
import numpy as np
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
from sklearn.linear_model import LinearRegression
model = LinearRegression(n_jobs=-1, normalize=False, fit_intercept=True)
# n_jobs doesn't speed up because I only have one target (Purchase)
#TODO, add sample_weight to fit() higher weights for underrepresented prod cat

# intercept = csr_matrix( np.ones((Gender_coded.shape[0],1))*1 )
# X = Prod_coded.toarray()
# X = np.concatenate(( np.ones((Gender_coded.shape[0],1)) , Prod_coded.toarray() ), axis=1)
# X = hstack( (Gender_coded,\
#              Age_coded,\
#              Occ_coded,\
#              City_coded,\
#              Years_Current_City_csr,\
#              Mar_status,\
#              ProdID_coded,\
#              User_coded,\
#              Prod_coded ))
# X = hstack( ( Gender_coded,\
#              Age_Marital_City_coded,\
#              Years_Current_City_csr,\
#              Occ_coded,\
#              User_coded,\
#              ProdID_coded,\
#              Prod_coded ))
X = hstack( ( User_coded,\
             ProdID_coded,\
             Prod_coded ))
Years_Current_City_poly
y = df.Purchase.values
model.fit(X, y)
model.intercept_, model.coef_[0], model.coef_.shape, X.shape

In [ ]:
from sklearn.metrics import r2_score
y_pred = model.predict(X)
print('Chi-Square = {:.4f}'.format(r2_score(y, y_pred)))

from sklearn.metrics import mean_squared_error
print('MSE = {:.4f}'.format(mean_squared_error(y, y_pred)))
print('RMSE = {:.4f}'.format(np.sqrt(mean_squared_error(y, y_pred))))

In [ ]:
X.shape
y.shape
X.shape[0]
# intercept.shape
# Prod_coded.shape

# Am I overfitting?

No, I'm underfitting!

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

def plot_learning_curve(model,X,y):
    # split data/target in train and test sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
    train_errors, val_errors = [], []
    train_sizes = np.linspace(1, X_train.shape[0], 100, dtype='int64')
    for m in train_sizes:
        # train with m samples
        model.fit(X_train[:m], y_train[:m])
        # predict training data and val data
        y_train_pred = model.predict(X_train[:m])
        y_val_pred = model.predict(X_val)
        # save MSE metric
        train_errors.append(mean_squared_error(y_train[:m], y_train_pred))
        val_errors.append(mean_squared_error(y_val, y_val_pred))
    plt.plot(train_sizes, np.sqrt(train_errors), 'r-+', linewidth=2, label='train')
    plt.plot(train_sizes, np.sqrt(val_errors), 'b-', linewidth=2, label='validation')

plot_learning_curve(model,X,y)

# Linear Regression (SGD version)
SGD: Stochastic Gradient Descent

In [ ]:
from sklearn.linear_model import SGDRegressor
model_SGD = SGDRegressor(max_iter=50, penalty=None, eta0=0.1, tol=1e-3, shuffle=True)
model_SGD.fit(X, y)

In [ ]:
y_predSGD = model_SGD.predict(X)
from sklearn.metrics import mean_squared_error
print('MSE = {:.4f}'.format(mean_squared_error(y, y_predSGD)))
print('RMSE = {:.4f}'.format(np.sqrt(mean_squared_error(y, y_predSGD))))

In [ ]:
model_SGD.n_iter_

# Test results Linear Regression
Run algo on test data to submit answer.<br>
Apply previous encoders to run model on test data set.

In [ ]:
filename = './test_HujdGe7/test.csv'
df_test = pd.read_csv(filename)

from scipy.sparse import csr_matrix
Years_Current_City_t = df_test.loc[:,'Stay_In_Current_City_Years'].apply(lambda x: 4 if x == '4+' else int(x)).values
Years_Current_City_t = csr_matrix(Years_Current_City_t.reshape(-1,1))
User_coded_t = User_encoder.transform(df_test.User_ID.values)
ProdID_coded_t = ProdID_encoder.transform(df_test.Product_ID.values)
# combine Prod_1, Prod_2 and Prod_3
temp_t = df_test.apply(lambda x:\
            '-'.join([str(x['Product_Category_1']),\
                      str(x['Product_Category_2']), \
                      str(x['Product_Category_3']), \
                     ]), axis=1 )
Prod_coded_t = Prod_encoder.transform(temp_t.values)
# combine age, marital status and city
temp_Age_Marital_City_t = df_test.apply(lambda x:\
            '-'.join([str(x['Age']),\
                      str(x['Marital_Status']), \
                      str(x['City_Category']), \
                     ]), axis=1 )
Age_Marital_City_coded_t = Age_Marital_City_encoder.transform(temp_Age_Marital_City_t.values)
Occ_coded_t = Occ_encoder.transform(df_test.Occupation.values.reshape(-1,1))
Gender_coded_t = Gender_encoder.transform(df_test.Gender.values)
Gender_coded_t = csr_matrix(Gender_coded_t.reshape(-1,1))
Intercept_t = csr_matrix(np.ones( (Gender_coded_t.shape[0],1) ) )

# X_test = hstack( ( Years_Current_City_t,\
#              Gender_coded_t,\
#              Age_Marital_City_coded_t,\
#              Occ_coded_t,\
#              User_coded_t,\
#              ProdID_coded_t,\
#              Prod_coded_t ) )
X_test = hstack( (User_coded_t,\
             ProdID_coded_t,\
             Prod_coded_t ) )

y_pred_test = model.predict(X_test)

In [ ]:
df_results = df_test.loc[:,('User_ID','Product_ID')]
df_results['Purchase'] = y_pred_test.reshape(-1,1)
df_results.to_csv('Submission.csv', index=False)

# Correlation
Let's see what correlation between features tells us.

In [ ]:
# start with a dataframe copy
df_cpy = df.copy()
# one needs to encode categorical values first
# use scikit-learn encoder
from sklearn.preprocessing import LabelEncoder
Gender = LabelEncoder()
df_cpy.Gender = Gender.fit_transform(df.Gender.values)
Age = LabelEncoder()
df_cpy.Age = Age.fit_transform(df.Age.values)
City = LabelEncoder()
df_cpy.City_Category = City.fit_transform(df.City_Category.values)
Years = LabelEncoder()
df_cpy.Stay_In_Current_City_Years = Years.fit_transform(df.Stay_In_Current_City_Years.values)
P_ID = LabelEncoder()
df_cpy.Product_ID = P_ID.fit_transform(df.Product_ID.values)
U_ID = LabelEncoder()
df_cpy.User_ID = U_ID.fit_transform(df.User_ID.values)
# show correlation
df_cpy.corr()

Product Category 1 does help the most.  Product ID is second.<br>
If you want to use Cat 2 and Cat3 then combine them with Cat1 as they are somewhat correlated.<br>
Other features are expected to help just a little.<br>

Age and Marital status is 31% correlated. <br>
Age and City category is 12% correlated. <br>
Gender and Occupation is 11% correlated.<br>



## Correlation again after feature extraction

# Polynomial model

To generate a non-linear model in order to improve our model accuracy, we can use scikit-learn PolynomialFeatures method.
It needs a dense matrix. Our feature matrix is sparse otherwise we will run out of memory. Thus it is not possible (at least on my machine) to generate polynomial features on all features.
It doesn't help to create polynomes on categorical features when we use one-hot encoding.<br><br>
Only years in current city is numeric (i.e. not categorical) I tried adding polynomes of years in current city. No improvement on metrics, it even decreased performance when degree > 3.

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures
# # object to compute 
# poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
# # compute polynomial features
# X_poly = poly.fit_transform(X)
# model.fit(X_poly, y)
# y_pred = model.predict(X_poly)
# print('{:.4f}'.format(r2_score(y, y_pred)))

# Ridge Regularization
Learning curves shows there is no overfit. So there shouldn't be an improvement by adding regularization.

# Tree
Use tree instead of linear Regression

#### Modules version

In [ ]:
import sys
# sys.version_info for version formatted
print('Python version = {}'.format(sys.version.split('|', maxsplit=1)[0].strip()))
print('Pandas version = {}'.format(pd.__version__))
print('NumPy version = {}'.format(np.__version__))
print('Matplotlib version = {}'.format(sys.modules[plt.__package__].__version__))
skl = LabelEncoder.__module__.split('.', maxsplit=1)[0]
print('sci-kit learn version = {}'.format(sys.modules[skl].__version__))
sci = csr_matrix.__module__.split('.', maxsplit=1)[0]
print('SciPy version = {}'.format(sys.modules[sci].__version__))

## TODO
- check instances with highest error and lowest error
- check correlation after feature extraction
- correct for imbalance of instances based on Product ID 
- try decision tree
- try adding age as contineous feature (given as categorical)